In [17]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [18]:
import torch
print(torch.cuda.is_available())

True


In [19]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models
model, tokenizer = FastLanguageModel.from_pretrained( 
    model_name = "D:/Users/Groh_Justin/PycharmProjects/LLM/models/Meta-Llama-3-70B-Instruct/original", # --> unsloth/Meta-Llama-3.1-8B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...",
)

==((====))==  Unsloth 2025.1.7: Fast Llama patching. Transformers: 4.47.0.
   \\   /|    GPU: NVIDIA GeForce RTX 4080 SUPER. Max memory: 15.992 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.3.1+cu121. CUDA: 8.9. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [20]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none",
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

<a name="Data"></a>
### Data Prep

In [22]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset_path = "dataset_Malware/malware_analysis_dataset_2.json"  # alpaca_data_cleaned.json, malware_analysis_dataset_2.json
dataset = load_dataset("json", data_files=dataset_path, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/366 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model

In [23]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 1,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map:   0%|          | 0/366 [00:00<?, ? examples/s]

In [24]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4080 SUPER. Max memory = 15.992 GB.
11.326 GB of memory reserved.


In [25]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 366 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.796400
2,1.702900
3,1.772700
4,1.734900
5,1.627700
6,1.448800
7,1.294600
8,1.193300
9,1.096400
10,1.008100


In [26]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

307.6364 seconds used for training.
5.13 minutes used for training.
Peak reserved memory = 11.326 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 70.823 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model


In [28]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Analyze the provided memory dump from a system infected with malware. Identify and explain the key activities and behaviors of the malware based on the Volatility outputs for pstree, malfind, and netscan.",  # instruction
            "{\"pstree\": {\"Audit\": \"\\\\Device\\\\HarddiskVolume3\\\\Users\\\\BA-LK\\\\Documents\\\\3a5df5e2ea2cab2de87bbaa4c3e20684daf50245cf6738c5124fcb848a4b2827.exe\", \"Cmd\": \"\\\"C:\\\\Users\\\\BA-LK\\\\Documents\\\\3a5df5e2ea2cab2de87bbaa4c3e20684daf50245cf6738c5124fcb848a4b2827.exe\\\" \", \"CreateTime\": \"2024-11-20T06:37:10\", \"ImageFileName\": \"3a5df5e2ea2cab\", \"PID\": 1996, \"PPID\": 5200, \"Path\": \"C:\\\\Users\\\\BA-LK\\\\Documents\\\\3a5df5e2ea2cab2de87bbaa4c3e20684daf50245cf6738c5124fcb848a4b2827.exe\", \"SessionId\": 1, \"Threads\": 8, \"Wow64\": true}, \"malfind\": [{\"PID\": 1996, \"Process\": \"3a5df5e2ea2cab\", \"Protection\": \"PAGE_EXECUTE_READWRITE\", \"Start VPN\": 2031616, \"End VPN\": 2035711, \"Notes\": null}, {\"PID\": 1996, \"Process\": \"3a5df5e2ea2cab\", \"Protection\": \"PAGE_EXECUTE_READWRITE\", \"Start VPN\": 4849664, \"End VPN\": 4939775, \"Notes\": null}, {\"PID\": 1996, \"Process\": \"3a5df5e2ea2cab\", \"Protection\": \"PAGE_EXECUTE_READWRITE\", \"Start VPN\": 4194304, \"End VPN\": 4288511, \"Notes\": \"MZ header\"}, {\"PID\": 1996, \"Process\": \"3a5df5e2ea2cab\", \"Protection\": \"PAGE_EXECUTE_READWRITE\", \"Start VPN\": 35323904, \"End VPN\": 35446783, \"Notes\": null}, {\"PID\": 1996, \"Process\": \"3a5df5e2ea2cab\", \"Protection\": \"PAGE_EXECUTE_READWRITE\", \"Start VPN\": 35192832, \"End VPN\": 35287039, \"Notes\": null}, {\"PID\": 1996, \"Process\": \"3a5df5e2ea2cab\", \"Protection\": \"PAGE_EXECUTE_READWRITE\", \"Start VPN\": 35454976, \"End VPN\": 35549183, \"Notes\": \"MZ header\"}], \"netscan\": [{\"Created\": \"2024-11-20T06:39:47\", \"ForeignAddr\": \"91.121.84.204\", \"ForeignPort\": 8080, \"LocalAddr\": \"192.168.2.132\", \"LocalPort\": 49864, \"Owner\": \"3a5df5e2ea2cab\", \"PID\": 1996, \"Proto\": \"TCPv4\", \"State\": \"SYN_SENT\"}, {\"Created\": \"2024-11-20T06:39:21\", \"ForeignAddr\": \"91.121.84.204\", \"ForeignPort\": 8080, \"LocalAddr\": \"192.168.2.132\", \"LocalPort\": 49856, \"Owner\": \"3a5df5e2ea2cab\", \"PID\": 1996, \"Proto\": \"TCPv4\", \"State\": \"CLOSED\"}]}",
            # input
            "",  # output - leave this blank for generation!
        )
    ], return_tensors="pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=2048, use_cache=True)
#outputs = model.generate(
#    **inputs, 
#    max_new_tokens=512, 
#    use_cache=True, 
#    temperature=0.7,  # Zwischen 0 und 1, höher für kreativere Antworten
#    top_p=0.9  # Top-p sampling, bei dem die Wahrscheinlichkeit der Top-N-Tokens berücksichtigt wird
#)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nAnalyze the provided memory dump from a system infected with malware. Identify and explain the key activities and behaviors of the malware based on the Volatility outputs for pstree, malfind, and netscan.\n\n### Input:\n{"pstree": {"Audit": "\\\\Device\\\\HarddiskVolume3\\\\Users\\\\BA-LK\\\\Documents\\\\3a5df5e2ea2cab2de87bbaa4c3e20684daf50245cf6738c5124fcb848a4b2827.exe", "Cmd": "\\"C:\\\\Users\\\\BA-LK\\\\Documents\\\\3a5df5e2ea2cab2de87bbaa4c3e20684daf50245cf6738c5124fcb848a4b2827.exe\\" ", "CreateTime": "2024-11-20T06:37:10", "ImageFileName": "3a5df5e2ea2cab", "PID": 1996, "PPID": 5200, "Path": "C:\\\\Users\\\\BA-LK\\\\Documents\\\\3a5df5e2ea2cab2de87bbaa4c3e20684daf50245cf6738c5124fcb848a4b2827.exe", "SessionId": 1, "Threads": 8, "Wow64": true}, "malfind": [{"PID": 1996, "Process": "3a

 `TextStreamer` for continuous inference 

In [35]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Analyze the provided memory dump from a system where potentially non-malicious software was executed. Identify and explain the key details of the process based on the Volatility output for pstree.", # instruction
        "{\"pstree\": {\"Audit\": \"\\\\Device\\\\HarddiskVolume3\\\\Users\\\\BA-LK\\\\Documents\\\\4d42711b63f90ff9af3d6c8e4edb3ff08cab6fe5131d9a43f4d10d1ca51f7378.exe\", \"Cmd\": \"\\\\\"C:\\\\Users\\\\BA-LK\\\\Documents\\\\4d42711b63f90ff9af3d6c8e4edb3ff08cab6fe5131d9a43f4d10d1ca51f7378.exe\\\\\"\", \"CreateTime\": \"2024-11-19T16:51:36\", \"ExitTime\": null, \"Handles\": null, \"ImageFileName\": \"4d42711b63f90f\", \"Offset(V)\": 200137317482624, \"PID\": 5688, \"PPID\": 648, \"Path\": \"C:\\\\Users\\\\BA-LK\\\\Documents\\\\4d42711b63f90ff9af3d6c8e4edb3ff08cab6fe5131d9a43f4d10d1ca51f7378.exe\", \"SessionId\": 0, \"Threads\": 7, \"Wow64\": true, \"__children\": []}, \"netscan\": [{\"Created\": \"2024-11-19T16:51:36\", \"ForeignAddr\": \"0.0.0.0\", \"ForeignPort\": 0, \"LocalAddr\": \"0.0.0.0\", \"LocalPort\": 21, \"Offset\": 200137283453008, \"Owner\": \"4d42711b63f90f\", \"PID\": 5688, \"Proto\": \"TCPv4\", \"State\": \"LISTENING\", \"__children\": []}, {\"Created\": \"2024-11-19T16:51:36\", \"ForeignAddr\": \"::\", \"ForeignPort\": 0, \"LocalAddr\": \"::\", \"LocalPort\": 21, \"Offset\": 200137283453008, \"Owner\": \"4d42711b63f90f\", \"PID\": 5688, \"Proto\": \"TCPv6\", \"State\": \"LISTENING\", \"__children\": []}, {\"Created\": \"2024-11-19T16:51:36\", \"ForeignAddr\": \"0.0.0.0\", \"ForeignPort\": 0, \"LocalAddr\": \"0.0.0.0\", \"LocalPort\": 21, \"Offset\": 200137283457936, \"Owner\": \"4d42711b63f90f\", \"PID\": 5688, \"Proto\": \"TCPv4\", \"State\": \"LISTENING\", \"__children\": []}, {\"Created\": \"2024-11-19T16:51:36\", \"ForeignAddr\": \"0.0.0.0\", \"ForeignPort\": 0, \"LocalAddr\": \"127.0.0.1\", \"LocalPort\": 14147, \"Offset\": 200137283458288, \"Owner\": \"4d42711b63f90f\", \"PID\": 5688, \"Proto\": \"TCPv4\", \"State\": \"LISTENING\", \"__children\": []}, {\"Created\": \"2024-11-19T16:51:36\", \"ForeignAddr\": \"::\", \"ForeignPort\": 0, \"LocalAddr\": \"::1\", \"LocalPort\": 14147, \"Offset\": 200137283460400, \"Owner\": \"4d42711b63f90f\", \"PID\": 5688, \"Proto\": \"TCPv6\", \"State\": \"LISTENING\", \"__children\": []}]}", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

#_ = model.generate(
#    **inputs, 
#    streamer = text_streamer,
#    max_new_tokens=512, 
#    use_cache=True, 
#    temperature=0.7,  # Zwischen 0 und 1, höher für kreativere Antworten
#    top_p=0.9  # Top-p sampling, bei dem die Wahrscheinlichkeit der Top-N-Tokens berücksichtigt wird
#)

_ = model.generate(
    **inputs, 
    streamer = text_streamer,
    max_new_tokens=2048, 
    use_cache=True, 
)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Analyze the provided memory dump from a system where potentially non-malicious software was executed. Identify and explain the key details of the process based on the Volatility output for pstree.

### Input:
{"pstree": {"Audit": "\\Device\\HarddiskVolume3\\Users\\BA-LK\\Documents\\4d42711b63f90ff9af3d6c8e4edb3ff08cab6fe5131d9a43f4d10d1ca51f7378.exe", "Cmd": "\\"C:\\Users\\BA-LK\\Documents\\4d42711b63f90ff9af3d6c8e4edb3ff08cab6fe5131d9a43f4d10d1ca51f7378.exe\\"", "CreateTime": "2024-11-19T16:51:36", "ExitTime": null, "Handles": null, "ImageFileName": "4d42711b63f90f", "Offset(V)": 200137317482624, "PID": 5688, "PPID": 648, "Path": "C:\\Users\\BA-LK\\Documents\\4d42711b63f90ff9af3d6c8e4edb3ff08cab6fe5131d9a43f4d10d1ca51f7378.exe", "SessionId": 0, "Threads": 7, "Wow64": true, "__children": []}, "ne

<a name="Save"></a>
### Saving, loading finetuned models
**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", 
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) 

### Saving to float16 for VLLM

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if True: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )